# Run this code before handing off data to be analyzed. 

## It asks the user for the:
- treatment date (YYYY-MM-DD), the treatment time (HH:MM, in 24 hr time), 
- then the treatment type (e.g. Area X bilateral lesion, DOI 10 mg/kg). 
- Then, point it to the animal's data folder (e.g. USA5508's big folder of data). The code looks through the folders of .wav files from SAP to find the day they were created (as well as the different days the .wav files were created, if applicable) so we can add the year of the recordings to the .wav files, since SAP only saves the month, date, and time of the recording.

## RUN THIS CODE IF YOU'RE ON THE LAMBDA COMPUTER:

In [ ]:
from pathlib import Path
import time
import json
from datetime import datetime

# Set the path to the main folder here
# Example: folder_path = "/path/to/your/main/folder"
folder_path = "PASTE_YOUR_PATH_HERE"

# Function to validate user input for treatment date
def get_treatment_date():
    while True:
        treatment_date_input = input("Please enter the treatment date (YYYY-MM-DD): ")
        try:
            # Parse the input date to check if it's valid
            treatment_date = datetime.strptime(treatment_date_input, '%Y-%m-%d')
            return treatment_date.strftime('%Y-%m-%d')  # Return the date in the proper format
        except ValueError:
            print("Invalid date format. Please try again.")

# Function to validate user input for treatment time
def get_treatment_time():
    while True:
        treatment_time_input = input("Please enter the treatment time (HH:MM, 24-hour format): ")
        try:
            # Parse the input time to check if it's valid
            treatment_time = datetime.strptime(treatment_time_input, '%H:%M')
            return treatment_time.strftime('%H:%M')  # Return the time in the proper format
        except ValueError:
            print("Invalid time format. Please try again.")

# Function to get treatment type
def get_treatment_type():
    treatment_type = input("Please enter the treatment type: ")
    return treatment_type

# Get the treatment date, time, and type from the user
treatment_date = get_treatment_date()
treatment_time = get_treatment_time()
treatment_type = get_treatment_type()

# Check if a folder path was provided and if it exists
main_folder = Path(folder_path)
if main_folder.is_dir():
    # Dictionary to store subdirectory info
    folder_data = {
        "treatment_date": treatment_date,  # Store the treatment date entered by the user
        "treatment_time": treatment_time,  # Store the treatment time entered by the user
        "treatment_type": treatment_type,  # Store the treatment type entered by the user
        "subdirectories": {}
    }

    # Loop through each subdirectory in the main folder
    for subdirectory in main_folder.iterdir():
        if subdirectory.is_dir():  # Ensure it is a subdirectory
            # Get the last modification time of the subdirectory
            subdirectory_mod_time = subdirectory.stat().st_mtime
            subdirectory_mod_date = time.strftime('%Y-%m-%d', time.localtime(subdirectory_mod_time))

            # Set to store unique file modification dates within the subdirectory
            unique_mod_dates = set()

            # Loop through files within this subdirectory
            for file in subdirectory.rglob('*'):
                if file.is_file():
                    # Get the modification time of the file
                    file_mod_time = file.stat().st_mtime
                    
                    # Convert the timestamp to a date (not including time)
                    file_mod_date = time.strftime('%Y-%m-%d', time.localtime(file_mod_time))
                    
                    # Add the file modification date to the set
                    unique_mod_dates.add(file_mod_date)
            
            # Add the subdirectory modification date and file modification dates to the dictionary
            folder_data["subdirectories"][subdirectory.name] = {
                "subdirectory_mod_date": subdirectory_mod_date,
                "unique_file_mod_dates": sorted(list(unique_mod_dates))  # Convert set to a sorted list
            }

    # Save the data to a .json file with the main folder's name
    json_file_name = f"{main_folder.name}_modification_data.json"
    json_file_path = main_folder / json_file_name

    with open(json_file_path, 'w') as json_file:
        json.dump(folder_data, json_file, indent=4)

    print(f"Data has been saved to {json_file_path}")
else:
    print("The specified path is not a valid directory.")


## Running on a Mac/PC OPtion 1: If you run this cell, the will ask the the treatment date,time, and type, and .wav folder path for ONE bird. It will use a file explorer pop-up window for you to select the folder with the bird's data.

In [1]:
from pathlib import Path
import time
import json
from tkinter import Tk
from tkinter import filedialog
from datetime import datetime

# Function to validate user input for treatment date
def get_treatment_date():
    while True:
        treatment_date_input = input("Please enter the treatment date (YYYY-MM-DD): ")
        try:
            # Parse the input date to check if it's valid
            treatment_date = datetime.strptime(treatment_date_input, '%Y-%m-%d')
            return treatment_date.strftime('%Y-%m-%d')  # Return the date in the proper format
        except ValueError:
            print("Invalid date format. Please try again.")

# Function to validate user input for treatment time
def get_treatment_time():
    while True:
        treatment_time_input = input("Please enter the treatment time (HH:MM, 24-hour format): ")
        try:
            # Parse the input time to check if it's valid
            treatment_time = datetime.strptime(treatment_time_input, '%H:%M')
            return treatment_time.strftime('%H:%M')  # Return the time in the proper format
        except ValueError:
            print("Invalid time format. Please try again.")

# Function to get treatment type
def get_treatment_type():
    treatment_type = input("Please enter the treatment type: ")
    return treatment_type

# Get the treatment date, time, and type from the user
treatment_date = get_treatment_date()
treatment_time = get_treatment_time()
treatment_type = get_treatment_type()

# Initialize the folder_path variable
folder_path = []

# If folder_path is empty, open a GUI to let the user select a folder
if not folder_path:
    # Hide the main Tkinter window
    Tk().withdraw()
    
    # Open a dialog for folder selection
    folder_path = filedialog.askdirectory(title="Select Main Folder")

# Check if a folder was selected
if folder_path:
    # Path object for the selected folder
    main_folder = Path(folder_path)

    # Dictionary to store subdirectory info
    folder_data = {
        "treatment_date": treatment_date,  # Store the treatment date entered by the user
        "treatment_time": treatment_time,  # Store the treatment time entered by the user
        "treatment_type": treatment_type,  # Store the treatment type entered by the user
        "subdirectories": {}
    }

    # Loop through each subdirectory in the main folder
    for subdirectory in main_folder.iterdir():
        if subdirectory.is_dir():  # Ensure it is a subdirectory
            # Get the creation (birth) date of the subdirectory
            subdirectory_creation_time = subdirectory.stat().st_birthtime
            subdirectory_creation_date = time.strftime('%Y-%m-%d', time.localtime(subdirectory_creation_time))

            # Set to store unique file creation (birth) dates within the subdirectory
            unique_birth_dates = set()

            # Loop through files within this subdirectory
            for file in subdirectory.rglob('*'):
                if file.is_file():
                    # Get the creation (birth) time of the file
                    file_creation_time = file.stat().st_birthtime
                    
                    # Convert the timestamp to a date (not including time)
                    file_creation_date = time.strftime('%Y-%m-%d', time.localtime(file_creation_time))
                    
                    # Add the file creation date to the set
                    unique_birth_dates.add(file_creation_date)
            
            # Add the subdirectory creation date and file creation dates to the dictionary
            folder_data["subdirectories"][subdirectory.name] = {
                "subdirectory_creation_date": subdirectory_creation_date,
                "unique_file_creation_dates": sorted(list(unique_birth_dates))  # Convert set to a sorted list
            }

    # Save the data to a .json file with the main folder's name
    json_file_name = f"{main_folder.name}_creation_data.json"
    json_file_path = main_folder / json_file_name

    with open(json_file_path, 'w') as json_file:
        json.dump(folder_data, json_file, indent=4)

    print(f"Data has been saved to {json_file_path}")
else:
    print("No folder was selected.")


Data has been saved to /Users/mirandahulsey-vincent/Desktop/sample_wav_files/sample_wav_files_creation_data.json


## Running on a Mac/PC Option 2: If you run this cell, it will do the same as the cell above (ask for the treatment day, time, and type for ONE bird) but, instead of using the file exporer pop-up window, it will have you write out the path to the folder containing the bird's recordings in line 8 (folder_path).

In [3]:
from pathlib import Path
import time
import json
from datetime import datetime

# Set the path to the main folder here
# Example: folder_path = '/path/to/your/main/folder'
folder_path = '/Users/mirandahulsey-vincent/Desktop/sample_wav_files'

# Function to validate user input for treatment date
def get_treatment_date():
    while True:
        treatment_date_input = input("Please enter the treatment date (YYYY-MM-DD): ")
        try:
            # Parse the input date to check if it's valid
            treatment_date = datetime.strptime(treatment_date_input, '%Y-%m-%d')
            return treatment_date.strftime('%Y-%m-%d')  # Return the date in the proper format
        except ValueError:
            print("Invalid date format. Please try again.")

# Function to validate user input for treatment time
def get_treatment_time():
    while True:
        treatment_time_input = input("Please enter the treatment time (HH:MM, 24-hour format): ")
        try:
            # Parse the input time to check if it's valid
            treatment_time = datetime.strptime(treatment_time_input, '%H:%M')
            return treatment_time.strftime('%H:%M')  # Return the time in the proper format
        except ValueError:
            print("Invalid time format. Please try again.")

# Function to get treatment type
def get_treatment_type():
    treatment_type = input("Please enter the treatment type: ")
    return treatment_type

# Get the treatment date, time, and type from the user
treatment_date = get_treatment_date()
treatment_time = get_treatment_time()
treatment_type = get_treatment_type()

# Check if a folder path was provided and if it exists
main_folder = Path(folder_path)
if main_folder.is_dir():
    # Dictionary to store subdirectory info
    folder_data = {
        "treatment_date": treatment_date,  # Store the treatment date entered by the user
        "treatment_time": treatment_time,  # Store the treatment time entered by the user
        "treatment_type": treatment_type,  # Store the treatment type entered by the user
        "subdirectories": {}
    }

    # Loop through each subdirectory in the main folder
    for subdirectory in main_folder.iterdir():
        if subdirectory.is_dir():  # Ensure it is a subdirectory
            # Get the creation (birth) date of the subdirectory
            subdirectory_creation_time = subdirectory.stat().st_birthtime
            subdirectory_creation_date = time.strftime('%Y-%m-%d', time.localtime(subdirectory_creation_time))

            # Set to store unique file creation (birth) dates within the subdirectory
            unique_birth_dates = set()

            # Loop through files within this subdirectory
            for file in subdirectory.rglob('*'):
                if file.is_file():
                    # Get the creation (birth) time of the file
                    file_creation_time = file.stat().st_birthtime
                    
                    # Convert the timestamp to a date (not including time)
                    file_creation_date = time.strftime('%Y-%m-%d', time.localtime(file_creation_time))
                    
                    # Add the file creation date to the set
                    unique_birth_dates.add(file_creation_date)
            
            # Add the subdirectory creation date and file creation dates to the dictionary
            folder_data["subdirectories"][subdirectory.name] = {
                "subdirectory_creation_date": subdirectory_creation_date,
                "unique_file_creation_dates": sorted(list(unique_birth_dates))  # Convert set to a sorted list
            }

    # Save the data to a .json file with the main folder's name
    json_file_name = f"{main_folder.name}_creation_data.json"
    json_file_path = main_folder / json_file_name

    with open(json_file_path, 'w') as json_file:
        json.dump(folder_data, json_file, indent=4)

    print(f"Data has been saved to {json_file_path}")
else:
    print("The specified path is not a valid directory.")


Data has been saved to /Users/mirandahulsey-vincent/Desktop/sample_wav_files/sample_wav_files_creation_data.json
